In [124]:
#imports
import pandas as pd 
from pandas import DataFrame
from slugify import slugify
import requests
from tqdm.notebook import tqdm
from time import sleep
from bs4 import BeautifulSoup as bs
import json
from datetime import datetime
import webbrowser
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import numpy as np

In [125]:
#getting both csv files 
movie_details = pd.read_csv('/Users/armaan/src/bbox/mdata/thenum/movie_details.csv') 
imdb_movies = pd.read_csv('/Users/armaan/src/bbox/mdata/imdb/imdb_movies.csv') 
#converting names to lower and some cols to int
movie_details['name_lower'] = movie_details['name'].str.lower()
imdb_movies['year'] = pd.to_numeric(imdb_movies.startYear, errors='coerce')
imdb_movies['primaryTitle_lower'] = imdb_movies['primaryTitle'].str.lower()
imdb_movies['runtimeMinutes'] = pd.to_numeric(imdb_movies.runtimeMinutes, errors='coerce')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [129]:
#normalization
def normalize(x):
    if x:
        #for star wars
        x = x.replace('-ep-', '-episode-')
        #for harry potter
        x = x.replace('part-ii', 'part-2')
        x = x.replace('part-i', 'part-1')
        #for e.t. only!
        if x.startswith('et-'):
            x = x.replace('et-', 'e-t-')
        #for rise of skywalker only!
        if x == 'star-wars-the-rise-of-skywalker':
            x = "star-wars-episode-ix-the-rise-of-skywalker"
    return x

#find duplicates
def duplicate_scraper(imdb_id):
    r = requests.get("https://www.imdb.com/title/{}/".format(imdb_id))
    soup = bs(r.text)
    box_office = soup.find("div",{"id": "titleDetails"})
    all_h4 = box_office.findAll("h4")
    drop = True
    for h4 in all_h4:
        if h4.string == "Gross USA:":
            drop = False
            break
    return drop

In [127]:
#sluggifying and normalizing all names
movie_details['name_slug'] = movie_details.name_lower.apply(slugify)
imdb_movies['name_slug'] = imdb_movies.primaryTitle_lower.apply(slugify)
movie_details['name_slug'] = movie_details.name_slug.apply(normalize)
#creating new df with updated values
new_df = pd.merge(movie_details, imdb_movies, how ='inner', right_on = ['name_slug', 'year'], left_on = ['name_slug', 'year'])

In [130]:
#find duplicates and get rid of them
duplicates = new_df[new_df.name.duplicated()]
for index, row in tqdm(duplicates.iterrows()):
    match = new_df[(new_df['name'] == row["name"]) & (new_df['year'] == row["year"])]
    if len(match) > 1:
        imdb_ids = match["tconst"]
        for imdb_id in imdb_ids:
            drop = duplicate_scraper(imdb_id)
            if drop == True:
                new_df.drop( new_df[ new_df['tconst'] == imdb_id ].index , inplace=True)

In [131]:
#drops
new_df.drop(["endYear", "isAdult", "titleType", "startYear", "primaryTitle_lower", "runtimeMinutes", "genres", "name_lower", "name_slug", "primaryTitle", "originalTitle"], axis=1, inplace = True)

In [132]:
new_df

,name,year,domestic_box_office,international_box_office,worldwide_box_office,opening_weekend,legs,production_budget,theater_counts,mpaa_rating,running_time,genre,production_method,creative_type,production_companies,synopsis,link,tconst
0,Star Wars Ep. VII: The Force Awakens,2015,936662225,1131561399.00,2068223624.00,247966675.00,3.78,306000000.00,9.20,PG-13,136.00,Adventure,Animation/Live Action,Science Fiction,Lucasfilm,"Rey, a scavenger from the planet Jakku, and re...",https://www.the-numbers.com/movie/Star-Wars-Ep...,tt2488496
1,Avengers: Endgame,2019,858373000,1939427564.00,2797800564.00,357115007.00,2.4,400000000.00,8.10,PG-13,181.00,Action,Animation/Live Action,Super Hero,Marvel Studios,The grave course of events set in motion by Th...,https://www.the-numbers.com/movie/Avengers-End...,tt4154796
2,Avatar,2009,760507625,2028193712.00,2788701337.00,77025481.00,9.87,237000000.00,13.20,PG-13,162.00,Action,Animation/Live Action,Science Fiction,Dune Entertainment,"Jake Sully is a wounded ex-marine, thrust into...",https://www.the-numbers.com/movie/Avatar#tab=s...,tt0499549
3,Black Panther,2018,700059566,646853595.00,1346913161.00,202003951.00,3.47,200000000.00,9.90,PG-13,120.00,Action,Live Action,Super Hero,Marvel Studios,"King T'Challa returns home to the reclusive, t...",https://www.the-numbers.com/movie/Black-Panthe...,tt1825683
4,Avengers: Infinity War,2018,678815482,1369544272.00,2048359754.00,257698183.00,2.63,300000000.00,7.70,PG-13,156.00,Action,Animation/Live Action,Super Hero,Marvel Studios,As the Avengers and their allies have continue...,https://www.the-numbers.com/movie/Avengers-Inf...,tt4154756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,Jumper,2008,80172128,142468684.00,222640812.00,27354808.00,2.93,82500000.00,5.00,PG-13,88.00,Adventure,Live Action,Science Fiction,Regency Enterprises,NaN,https://www.the-numbers.com/movie/Jumper#tab=s...,tt0489099
945,The Mummy,2017,80101125,329852780.00,409953905.00,31668375.00,2.53,195000000.00,3.70,PG-13,107.00,Adventure,Live Action,Historical Fiction,Universal Pictures,Thought safely entombed in a tomb deep beneath...,https://www.the-numbers.com/movie/Mummy-The-(2...,tt2345759
946,Jack Reacher,2012,80070736,137300000.00,217370736.00,15210156.00,5.26,60000000.00,5.00,PG-13,130.00,Drama,Live Action,Contemporary Fiction,Paramount Pictures,Six shots. Five dead. One heartland city throw...,https://www.the-numbers.com/movie/Jack-Reacher...,tt0790724
947,Goosebumps,2015,80069458,78835866.00,158905324.00,23618556.00,3.39,58000000.00,5.80,PG,103.00,Horror,Animation/Live Action,Fantasy,Columbia Pictures,Upset about moving from a big city to a small ...,https://www.the-numbers.com/movie/Goosebumps#t...,tt1051904


In [133]:
#get poster path and add it to new col in df
poster_link_list = []
for p in tqdm(new_df['tconst']):
    tmdb = requests.get("https://api.themoviedb.org/3/movie/{}?api_key=a6837e56522d89b354d01b07e393bebc&language=en-US".format(p))
    try:
        poster_path = tmdb.json()['poster_path']
        poster_link = "https://image.tmdb.org/t/p/w300_and_h450_bestv2/{}".format(poster_path)
    except:
        if p:
            print ("no poster for " + p)
    poster_link_list.append(poster_link)

new_df["poster_path"] = poster_link_list

In [138]:
bbox_df = new_df

bbox_df['bb_profit'] = bbox_df.worldwide_box_office - bbox_df.production_budget
bbox_df['bb_profit_multiple'] = bbox_df.worldwide_box_office/bbox_df.worldwide_box_office

bbox_df['bb_profit_score'] = (bbox_df.bb_profit / bbox_df.bb_profit.max()) * 100
bbox_df['bb_profit_multiple_score'] = (bbox_df.bb_profit_multiple / bbox_df.bb_profit_multiple.max()) * 100

bbox_df['bb_score'] = (bbox_df['bb_profit_score']*0.7 + bbox_df['bb_profit_multiple_score']*0.3)

In [139]:
#downloads df to csv
bbox_df.to_csv('/Users/armaan/src/bbox/mdata/final_result/master_table.csv', index=False)